<a href="https://colab.research.google.com/github/thravt/AIProjectsHomework/blob/main/HW2_Tyler_Thraves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Base imports/installs

In [1]:
!pip install opendatasets
import math
import pandas as pd
import matplotlib.pyplot as plt
import opendatasets as od
import numpy as np
import seaborn as sns
import random
import os
!pip install tensorflow_decision_forests
!pip install wurlitzer
# Keep using Keras 2
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import tensorflow_decision_forests as tfdf
import tensorflow as tf
import tf_keras

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-2.17.0:
  

Download the database from HW1

In [2]:
od.download(
    "https://www.kaggle.com/datasets/devzohaib/eligibility-prediction-for-loan")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tylerthraves
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/devzohaib/eligibility-prediction-for-loan


100%|██████████| 7.80k/7.80k [00:00<00:00, 5.48MB/s]

**Task 1 (30 points): Implement a Decision Tree Classifier for your classification problem. You
may use a built-in package to implement your classifier.**

To implement the decision tree I will be using the tensorflow_decision_forests package in order to make the tree. The package is well documented on their official website: https://www.tensorflow.org/decision_forests/tutorials/beginner_colab. The choices for the model are listed below:

In [ ]:
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]

I'll use CartModel for this task, as it is the primary method discussed in lecture 3.

Loading the dataset:

In [3]:
file =('eligibility-prediction-for-loan/\
Loan_Data.csv')
Database = pd.read_csv(file)
Database['Loan_Status'] = Database['Loan_Status'].replace({'Y': 1, 'N': 0})

<ipython-input-3-69faf7997abd>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Database['Loan_Status'] = Database['Loan_Status'].replace({'Y': 1, 'N': 0})


In [4]:
label = 'Loan_Status'

Convert to a tensorflow dataset:

In [5]:
Dataset = tfdf.keras.pd_dataframe_to_tf_dataset(Database, label = label)

In [ ]:
print(Dataset)

<_PrefetchDataset element_spec=({'Loan_ID': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Gender': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Married': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Dependents': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Education': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Self_Employed': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'ApplicantIncome': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'CoapplicantIncome': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'LoanAmount': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Loan_Amount_Term': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Credit_History': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Property_Area': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


Make the Cart model:

In [ ]:
Cart1 = tfdf.keras.CartModel()
Cart1.fit(Dataset)

Use /tmp/tmpehdkwnhl as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.510521. Found 614 examples.
Training model...
Model trained in 0:00:00.054352
Compiling model...
Model compiled.


In [ ]:
print(Cart1.summary())

Model: "cart_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	ApplicantIncome
	CoapplicantIncome
	Credit_History
	Dependents
	Education
	Gender
	LoanAmount
	Loan_Amount_Term
	Loan_ID
	Married
	Property_Area
	Self_Employed

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.    "Credit_History"  1.000000 ################
    2.     "Property_Area"  0.500000 ######
    3.   "ApplicantIncome"  0.344828 ###
    4. "CoapplicantIncome"  0.222222 
    5.        "LoanAmount"  0.175439 

Variable Importance: NUM_AS_ROOT:
    1. "Credit_History"  1.000000 

Variable Importance: NUM_NODES:
    1.   "ApplicantIncome"  4.000000 ################
    2. "Coapplica

Show a visual representation of the tree for easier understanding:

In [ ]:
tfdf.model_plotter.plot_model_in_colab(Cart1, tree_idx=0, max_depth=15)

As we can see, the default settings gave us an accuracy of around 84%, which compared with the last homework I did, I'd say it turned out pretty great. Focusing on the individual parameters used for decisions, Credit history seems to be the most important, as a low enough amount will result in a nearly guranteed denial. If it's high enough, it seems like a semiurban property area will result in a likely approval. From there it's mostly a balance of the applicant income, the loan amount, and the coapplicant income, with a high applicant income and coapplicant income being more likely to get approval, and a high loan amount being less likely to get approval.

**Visualize the decision tree structure for at least three different parameter settings.
Comment on how the depth and complexity change the tree.**

The first parameter setting to be visualized is the defaults, as shown above.

The second I will be visualizing is setting min_examples to 2, in order to see if the decreased requirement will result in more accurate leafs.

In [ ]:
Cart2 = tfdf.keras.CartModel(min_examples = 2)
Cart2.fit(Dataset)

Use /tmp/tmppyufub9g as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.251228. Found 614 examples.
Training model...
Model trained in 0:00:00.046405
Compiling model...
Model compiled.


In [ ]:
print(Cart2.summary())

Model: "cart_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	ApplicantIncome
	CoapplicantIncome
	Credit_History
	Dependents
	Education
	Gender
	LoanAmount
	Loan_Amount_Term
	Loan_ID
	Married
	Property_Area
	Self_Employed

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.    "Credit_History"  1.000000 ################
    2.     "Property_Area"  0.500000 ######
    3. "CoapplicantIncome"  0.342105 ###
    4.   "ApplicantIncome"  0.220339 #
    5.        "LoanAmount"  0.156627 
    6.        "Dependents"  0.146067 
    7.         "Education"  0.138298 

Variable Importance: NUM_AS_ROOT:
    1. "Credit_History"  1.000000 

Variable Importance: NUM

In [ ]:
tfdf.model_plotter.plot_model_in_colab(Cart2, tree_idx=0, max_depth=15)

This tree had the exact same accuracy at .84, but the logloss was less than half the default tree. While this does get leaf nodes with more purity, I'm not sure that many of the splits are worth it, as there's often splits that result in less than 10% of the dataset being split off, and the complexity this could give doesn't seem worth it. The tree's also somewhere around 50% deeper, so this doesn't seem like it would scale well.

For my third model, I'll set the max_depth to 5. I'm guessing it will just end up with a truncated version of the first model, but it is possible that the lower depth will force the model to split differently.

In [ ]:
Cart3 = tfdf.keras.CartModel(max_depth = 5)
Cart3.fit(Dataset)

Use /tmp/tmpjo2e5s04 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.837110. Found 614 examples.
Training model...
Model trained in 0:00:00.059245
Compiling model...
Model compiled.


In [ ]:
print(Cart3.summary())

Model: "cart_model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	ApplicantIncome
	CoapplicantIncome
	Credit_History
	Dependents
	Education
	Gender
	LoanAmount
	Loan_Amount_Term
	Loan_ID
	Married
	Property_Area
	Self_Employed

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "Credit_History"  1.000000 

Variable Importance: NUM_AS_ROOT:
    1. "Credit_History"  1.000000 

Variable Importance: NUM_NODES:
    1. "Credit_History"  1.000000 

Variable Importance: SUM_SCORE:
    1. "Credit_History" 77.371266 



Winner takes all: false
Out-of-bag evaluation: accuracy:0.824561 logloss:0.44777
Number of trees: 1
Total number of nodes: 3

Number of nodes 

In [ ]:
tfdf.model_plotter.plot_model_in_colab(Cart3, tree_idx=0, max_depth=15)

Well, I was right that it would end up as a truncated version of the first model, but I didn't expect it to go back to a single split. Comparing the visualizations, I noticed that the splits between depth 2 and 5 would either result in both having a higher percentage of a single class, or would have a lower purity than that of the original split. As such, the model must have chosen to prune those nodes.

Overall, the default settings for the model had the best balance of depth in my opinion. Lowering the minimum amount of nodes to split on did provide more pure splits, but those pure splits would barely have any examples in them, so I don't think those splits were worth the extra depth. Furthermore, it's hard to tell in that model if those splits were due to an actual good boundary, or if those entries were outliers.
Decreasing the maximum depth resulted in a model that has less depth and is less complex, but the simplicity could result in important decision points being lost.

**Do some research on what sensitivity analysis is and how it is performed (include
citations). Perform a sensitivity analysis to measure the impact of at least two input
features on your model's decision boundary.**

In order to learn about sensitivity analysis, I looked at this website: https://medium.com/@einat_93627/understand-your-black-box-model-using-sensitivity-analysis-practical-guide-ef6ac4175e55.

The way it's described is to change a feature in a database, and see how it changes the model. If it's a big change, the feature is important for the prediction. If it isn't, then the feature isn't very important.

The way I'll implement it for this is through permutation, in which I randomly shuffle the values for a feature.

There are also methods I didn't choose such as taking the uniform distribution, changing how missing values work, or to consider multiple variables at once.

As for the features that sensitivity analysis will be performed on, I'll choose credit history and applicant income, as those are the first 2 continuous values to cause splits.

Sensitivity analysis for credit history:

In [ ]:
SensitiveDatabase = Database.copy()
print(SensitiveDatabase)

      Loan_ID  Gender Married Dependents     Education Self_Employed  \
0    LP001002    Male      No          0      Graduate            No   
1    LP001003    Male     Yes          1      Graduate            No   
2    LP001005    Male     Yes          0      Graduate           Yes   
3    LP001006    Male     Yes          0  Not Graduate            No   
4    LP001008    Male      No          0      Graduate            No   
..        ...     ...     ...        ...           ...           ...   
609  LP002978  Female      No          0      Graduate            No   
610  LP002979    Male     Yes         3+      Graduate            No   
611  LP002983    Male     Yes          1      Graduate            No   
612  LP002984    Male     Yes          2      Graduate            No   
613  LP002990  Female      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0               5849                0.0         NaN           

In [ ]:
SensitiveDatabase['Credit_History'] = SensitiveDatabase['Credit_History'].sample(frac=1).reset_index(drop=True)

In [ ]:
print(Database['Credit_History'])

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
609    1.0
610    1.0
611    1.0
612    1.0
613    0.0
Name: Credit_History, Length: 614, dtype: float64


In [ ]:
print(SensitiveDatabase['Credit_History'])

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
609    0.0
610    1.0
611    1.0
612    1.0
613    1.0
Name: Credit_History, Length: 614, dtype: float64


In [ ]:
SensitiveDataset1 = tfdf.keras.pd_dataframe_to_tf_dataset(SensitiveDatabase, label = label)

In [ ]:
SensitiveCart1 = tfdf.keras.CartModel()
SensitiveCart1.fit(SensitiveDataset1)

Use /tmp/tmph4umjflk as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.279943. Found 614 examples.
Training model...
Model trained in 0:00:00.039118
Compiling model...
Model compiled.


In [ ]:
print(SensitiveCart1.summary())

Model: "cart_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	ApplicantIncome
	CoapplicantIncome
	Credit_History
	Dependents
	Education
	Gender
	LoanAmount
	Loan_Amount_Term
	Loan_ID
	Married
	Property_Area
	Self_Employed

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.     "Property_Area"  1.000000 ################
    2.  "Loan_Amount_Term"  0.500000 ######
    3.         "Education"  0.343750 ###
    4.   "ApplicantIncome"  0.268293 #
    5.        "LoanAmount"  0.186441 
    6.        "Dependents"  0.174603 
    7. "CoapplicantIncome"  0.166667 

Variable Importance: NUM_AS_ROOT:
    1. "Property_Area"  1.000000 

Variable Importance: NUM_

In [ ]:
tfdf.model_plotter.plot_model_in_colab(SensitiveCart1, tree_idx=0, max_depth=15)

As we can see, the new tree is quite different from the old one, and this means that as expected, credit history has a big impact on the decision boundary.

Sensitivity analysis for applicant income:

In [ ]:
SensitiveDatabase = Database.copy()

In [ ]:
SensitiveDatabase['ApplicantIncome'] = SensitiveDatabase['ApplicantIncome'].sample(frac=1).reset_index(drop=True)

In [ ]:
SensitiveDataset2 = tfdf.keras.pd_dataframe_to_tf_dataset(SensitiveDatabase, label = label)

In [ ]:
SensitiveCart2 = tfdf.keras.CartModel()
SensitiveCart2.fit(SensitiveDataset2)

Use /tmp/tmpbqn7xs99 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.271259. Found 614 examples.
Training model...
Model trained in 0:00:00.040092
Compiling model...
Model compiled.


In [ ]:
print(SensitiveCart2.summary())

Model: "cart_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	ApplicantIncome
	CoapplicantIncome
	Credit_History
	Dependents
	Education
	Gender
	LoanAmount
	Loan_Amount_Term
	Loan_ID
	Married
	Property_Area
	Self_Employed

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.    "Credit_History"  1.000000 ################
    2.     "Property_Area"  0.500000 ######
    3.   "ApplicantIncome"  0.342105 ###
    4.        "LoanAmount"  0.191176 
    5. "CoapplicantIncome"  0.166667 
    6.  "Loan_Amount_Term"  0.151163 
    7.         "Education"  0.139785 
    8.           "Married"  0.139785 

Variable Importance: NUM_AS_ROOT:
    1. "Credit_History"

In [ ]:
tfdf.model_plotter.plot_model_in_colab(SensitiveCart2, tree_idx=0, max_depth=15)

This tree is pretty similar to the original tree earlier on, but the deeper it gets, the more different it is. This indicates that while it's not as important as credit history or property area for the decision boundary, it does still have a signifigant importance.

**Task 2 (30 points): From the Bagging and Boosting ensemble methods pick any one algorithm
from each category. Implement both the algorithms using the same data.**

For the bagging algorithm I will be random forest. Like before, this will be implemented through tensorflow decision forests.

In [ ]:
#Make the forest model
RandomForest = tfdf.keras.RandomForestModel()
RandomForest.fit(Dataset)

Use /tmp/tmp9r77w04n as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.070493. Found 614 examples.
Training model...
Model trained in 0:00:00.550988
Compiling model...
Model compiled.


Summary of model:

In [ ]:
print(RandomForest.summary())

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	ApplicantIncome
	CoapplicantIncome
	Credit_History
	Dependents
	Education
	Gender
	LoanAmount
	Loan_Amount_Term
	Loan_ID
	Married
	Property_Area
	Self_Employed

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.    "Credit_History"  0.454517 ################
    2.        "LoanAmount"  0.215920 ####
    3.   "ApplicantIncome"  0.207329 ####
    4.     "Property_Area"  0.201034 ####
    5. "CoapplicantIncome"  0.187291 ###
    6.  "Loan_Amount_Term"  0.145994 #
    7.        "Dependents"  0.139434 #
    8.           "Married"  0.134909 
    9.         "Education"  0.128134 
   10

Plotting any individual tree for this model wouldn't be helpful due to the injected noise. The oob evaluation gave an accuracy of around 80%, and logloss of around 0.51, so the model doesn't seem as good as the Cart model.

For the Boosting model, I'll be using a Gradient boosting model, tensorflow's GradientBoostedTreesModel.

In [ ]:
GradientBoost = tfdf.keras.GradientBoostedTreesModel()
GradientBoost.fit(Dataset)

Use /tmp/tmpw8bjseq_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.278482. Found 614 examples.
Training model...
Model trained in 0:00:00.184444
Compiling model...
Model compiled.


Summary of the model:

In [ ]:
print(GradientBoost.summary())

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	ApplicantIncome
	CoapplicantIncome
	Credit_History
	Dependents
	Education
	Gender
	LoanAmount
	Loan_Amount_Term
	Loan_ID
	Married
	Property_Area
	Self_Employed

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.    "Credit_History"  1.000000 ################
    2.        "LoanAmount"  0.275070 #
    3.  "Loan_Amount_Term"  0.229720 
    4.     "Property_Area"  0.228862 
    5.   "ApplicantIncome"  0.228088 
    6. "CoapplicantIncome"  0.216107 
    7.           "Married"  0.195554 
    8.        "Dependents"  0.184586 
    9.     "Self_Employed"  0.183242 
   

The gradient boost gave me a train-loss of 0.53, and a valid-loss of 1.02, so not as good as the random forest. The train-accuracy was 0.887, and valid-accuracy was .789, which was relatively comparable to random forest's .798. Overall I'm surprised that the train-accuracy for for gradient boost was only somewhat higher than for CART, but I suppose those small improvements can mean a lot with more data. Overall I feel that gradient performed the best.

**Use stratified k-fold cross-validation with at least three different folds (e.g., 5, 10, 15).
You may do your own research on this technique (include citations).**

Stratified k-fold validation involves splitting the dataset into k equally subsets, in which the subsets all share the same proportion of y values as the original set. Each fold gets to act as the test set once, while all others are the training set. The average scores between the subsets are then taken for the final score.

In order to learn this, I used this website: https://medium.com/@juanc.olamendy/a-comprehensive-guide-to-stratified-k-fold-cross-validation-for-unbalanced-data-014691060f17.

To implement stratified k-fold validation, I will be using sklearn.model_selection.StratifiedKFold(), found on the scikit website https://scikit-learn.org/1.6/modules/generated/sklearn.model_selection.StratifiedKFold.html. It would be simple enough for me to make a for loop with a modular arithmatic check to divide the data into groups, but since that would require around 60 datasets (30 train and 30 test) per model, I figured it would be more storage and time efficient to use this. I will be using 5 fold, 10 fold, and 15 fold (hence, the 30 datasets of each per model). In order to ensure the shuffle doesn't cause one model to do better, I'll use the random state 64.

First, I'm going to get this implemented for Random Forest.

In [6]:
from sklearn.model_selection import StratifiedKFold

In [7]:
skf5 = StratifiedKFold(n_splits=5, random_state = 64, shuffle = True)

In [8]:
X = Database.drop(label, axis = 1).values
y = Database[label].values

In [ ]:
rf5a = np.zeros(5)
rf5r = np.zeros(5)
rf5p = np.zeros(5)
for i, (train_index, test_index) in enumerate(skf5.split(X, y)):
  TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
  TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)

  RandomForestk = tfdf.keras.RandomForestModel()
  RandomForestk.fit(TrainDatak)
  RandomForestk.compile(metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
  evaluation = RandomForestk.evaluate(TestDatak)
  rf5a[i] = evaluation[1]
  rf5r[i] = evaluation[2]
  rf5p[i] = evaluation[3]
rf5accuracy = np.mean(rf5a)
rf5recall = np.mean(rf5r)
rf5precision= np.mean(rf5p)
print("accuracy:", rf5accuracy)
print("recall:", rf5recall)
print("precision", rf5precision)

Use /tmp/tmpwlza_zu9 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.287496. Found 491 examples.
Training model...
Model trained in 0:00:00.358014
Compiling model...
Model compiled.
1/1 [==============================] - 1s 538ms/step - loss: 0.0000e+00 - accuracy: 0.8211 - recall_7: 0.9412 - precision_4: 0.8247
Use /tmp/tmp8r6uqg__ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.292609. Found 491 examples.
Training model...
Model trained in 0:00:00.358906
Compiling model...
Model compiled.
1/1 [==============================] - 0s 453ms/step - loss: 0.0000e+00 - accuracy: 0.7480 - recall_8: 0.9412 - precision_5: 0.7547
Use /tmp/tmpgfjzza4r as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.369073. Found 491 examples.
Training model...
Model trained in 0:00:00.432083
Compiling model...
Model compiled.
1/1 [==============================] - 0s 422ms/step - 

In [ ]:
skf10 = StratifiedKFold(n_splits=10, random_state = 64, shuffle = True)

In [ ]:
rf10a = np.zeros(10)
rf10r = np.zeros(10)
rf10p = np.zeros(10)
for i, (train_index, test_index) in enumerate(skf10.split(X, y)):
  TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
  TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)

  RandomForestk = tfdf.keras.RandomForestModel()
  RandomForestk.fit(TrainDatak)
  RandomForestk.compile(metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
  evaluation = RandomForestk.evaluate(TestDatak)
  rf10a[i] = evaluation[1]
  rf10r[i] = evaluation[2]
  rf10p[i] = evaluation[3]
rf10accuracy = np.mean(rf10a)
rf10recall = np.mean(rf10r)
rf10precision= np.mean(rf10p)
print("accuracy:", rf10accuracy)
print("recall:", rf10recall)
print("precision", rf10precision)

Use /tmp/tmp4i3mtcue as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.302973. Found 552 examples.
Training model...
Model trained in 0:00:00.411321
Compiling model...
Model compiled.
1/1 [==============================] - 1s 553ms/step - loss: 0.0000e+00 - accuracy: 0.8710 - recall_12: 1.0000 - precision_9: 0.8431
Use /tmp/tmp9lbof7l_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.288436. Found 552 examples.
Training model...
Model trained in 0:00:00.506161
Compiling model...
Model compiled.
1/1 [==============================] - 0s 444ms/step - loss: 0.0000e+00 - accuracy: 0.8065 - recall_13: 0.9535 - precision_10: 0.8039
Use /tmp/tmpnv6dj4bv as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.267860. Found 552 examples.
Training model...
Model trained in 0:00:00.389160
Compiling model...
Model compiled.
1/1 [==============================] - 0s 487ms/step

In [ ]:
skf15 = StratifiedKFold(n_splits=15, random_state = 64, shuffle = True)

In [ ]:
rf15a = np.zeros(15)
rf15r = np.zeros(15)
rf15p = np.zeros(15)
for i, (train_index, test_index) in enumerate(skf15.split(X, y)):
  TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
  TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)

  RandomForestk = tfdf.keras.RandomForestModel()
  RandomForestk.fit(TrainDatak)
  RandomForestk.compile(metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
  evaluation = RandomForestk.evaluate(TestDatak)
  rf15a[i] = evaluation[1]
  rf15r[i] = evaluation[2]
  rf15p[i] = evaluation[3]
rf15accuracy = np.mean(rf15a)
rf15recall = np.mean(rf15r)
rf15precision= np.mean(rf15p)
print("accuracy:", rf15accuracy)
print("recall:", rf15recall)
print("precision", rf15precision)

Use /tmp/tmpogr89p6i as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.297752. Found 573 examples.
Training model...
Model trained in 0:00:00.483632
Compiling model...
Model compiled.
1/1 [==============================] - 0s 473ms/step - loss: 0.0000e+00 - accuracy: 0.8293 - recall_22: 1.0000 - precision_19: 0.8056
Use /tmp/tmp73ltaurg as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.320888. Found 573 examples.
Training model...
Model trained in 0:00:00.441680
Compiling model...
Model compiled.
1/1 [==============================] - 1s 563ms/step - loss: 0.0000e+00 - accuracy: 0.9024 - recall_23: 1.0000 - precision_20: 0.8788
Use /tmp/tmp3p0sfuzx as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.285216. Found 573 examples.
Training model...
Model trained in 0:00:00.511567
Compiling model...
Model compiled.
1/1 [==============================] - 0s 443ms/ste

Next, the same implementation will be done with gradient boost.

In [ ]:
gb5a = np.zeros(5)
gb5r = np.zeros(5)
gb5p = np.zeros(5)
for i, (train_index, test_index) in enumerate(skf5.split(X, y)):
  TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
  TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)

  GradientBoostk = tfdf.keras.GradientBoostedTreesModel()
  GradientBoostk.fit(TrainDatak)
  GradientBoostk.compile(metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
  evaluation = GradientBoostk.evaluate(TestDatak)
  gb5a[i] = evaluation[1]
  gb5r[i] = evaluation[2]
  gb5p[i] = evaluation[3]
gb5accuracy = np.mean(gb5a)
gb5recall = np.mean(gb5r)
gb5precision= np.mean(gb5p)
print("accuracy:", gb5accuracy)
print("recall:", gb5recall)
print("precision", gb5precision)

Use /tmp/tmp0h61jgu_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.530996. Found 491 examples.
Training model...
Model trained in 0:00:00.406636
Compiling model...
Model compiled.
1/1 [==============================] - 1s 745ms/step - loss: 0.0000e+00 - accuracy: 0.8130 - recall_37: 0.9294 - precision_34: 0.8229
Use /tmp/tmpg_zrey83 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.436719. Found 491 examples.
Training model...
Model trained in 0:00:00.181244
Compiling model...
Model compiled.
1/1 [==============================] - 1s 796ms/step - loss: 0.0000e+00 - accuracy: 0.7805 - recall_38: 0.9765 - precision_35: 0.7685
Use /tmp/tmpms5ds95p as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.480966. Found 491 examples.
Training model...
Model trained in 0:00:00.170704
Compiling model...
Model compiled.
1/1 [==============================] - 0s 472ms/ste

In [ ]:
gb10a = np.zeros(10)
gb10r = np.zeros(10)
gb10p = np.zeros(10)
for i, (train_index, test_index) in enumerate(skf10.split(X, y)):
  TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
  TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)

  GradientBoostk = tfdf.keras.GradientBoostedTreesModel()
  GradientBoostk.fit(TrainDatak)
  GradientBoostk.compile(metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
  evaluation = GradientBoostk.evaluate(TestDatak)
  gb10a[i] = evaluation[1]
  gb10r[i] = evaluation[2]
  gb10p[i] = evaluation[3]
gb10accuracy = np.mean(gb10a)
gb10recall = np.mean(gb10r)
gb10precision= np.mean(gb10p)
print("accuracy:", gb10accuracy)
print("recall:", gb10recall)
print("precision", gb10precision)

Use /tmp/tmpkqjh59gj as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.467507. Found 552 examples.
Training model...
Model trained in 0:00:00.513424
Compiling model...
Model compiled.
1/1 [==============================] - 0s 459ms/step - loss: 0.0000e+00 - accuracy: 0.8387 - recall_42: 0.9535 - precision_39: 0.8367
Use /tmp/tmp6vxbhkpw as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.299979. Found 552 examples.
Training model...
Model trained in 0:00:00.218522
Compiling model...
Model compiled.
1/1 [==============================] - 0s 458ms/step - loss: 0.0000e+00 - accuracy: 0.8387 - recall_43: 1.0000 - precision_40: 0.8113
Use /tmp/tmpfptu8m31 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.346558. Found 552 examples.
Training model...
Model trained in 0:00:00.208349
Compiling model...
Model compiled.
1/1 [==============================] - 1s 522ms/ste

In [ ]:
gb15a = np.zeros(15)
gb15r = np.zeros(15)
gb15p = np.zeros(15)
for i, (train_index, test_index) in enumerate(skf15.split(X, y)):
  TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
  TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)

  GradientBoostk = tfdf.keras.GradientBoostedTreesModel()
  GradientBoostk.fit(TrainDatak)
  GradientBoostk.compile(metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
  evaluation = GradientBoostk.evaluate(TestDatak)
  gb15a[i] = evaluation[1]
  gb15r[i] = evaluation[2]
  gb15p[i] = evaluation[3]
gb15accuracy = np.mean(gb15a)
gb15recall = np.mean(gb15r)
gb15precision= np.mean(gb15p)
print("accuracy:", gb15accuracy)
print("recall:", gb15recall)
print("precision", gb15precision)

Use /tmp/tmpzxr5lpq0 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.309473. Found 573 examples.
Training model...
Model trained in 0:00:00.201780
Compiling model...
Model compiled.
1/1 [==============================] - 1s 508ms/step - loss: 0.0000e+00 - accuracy: 0.8049 - recall_52: 0.9655 - precision_49: 0.8000
Use /tmp/tmpmqa48xtr as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.333135. Found 573 examples.
Training model...
Model trained in 0:00:00.233111
Compiling model...
Model compiled.
1/1 [==============================] - 0s 481ms/step - loss: 0.0000e+00 - accuracy: 0.9024 - recall_53: 1.0000 - precision_50: 0.8788
Use /tmp/tmpn01ea1c1 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.303490. Found 573 examples.
Training model...
Model trained in 0:00:00.281153
Compiling model...
Model compiled.
1/1 [==============================] - 0s 448ms/ste

Now to compare the metrics between the models under each fold:

For ease of comparison, Random Forest will be on the left, and Gradient Boost will be on the right

K = 5:

Accuracy: 0.7817406296730042 vs 0.7932026982307434, GB wins

Recall: 0.931232488155365 vs 0.957366943359375, GB wins

Precision: 0.7898211359977723 vs 0.7882573127746582, RF wins

K = 10:

Accuracy: 0.7932046532630921 vs 0.7930988848209382, RF wins

Recall: 0.9501107335090637 vs 0.952491682767868, GB wins

Precision: 0.7923055291175842 vs 0.7910798788070679, RF wins

K = 15:

Accuracy: 0.7997560977935791 vs 0.79158536195755, RF wins

Recall: 0.9571428537368775 vs 0.9500820954640706, RF wins

Precision: 0.7962838888168335 vs 0.7915743430455525, RF wins

The first observation is that RF always did better on precision, so if you want your positive predictions to be correct, it's the better model. As for accuracy and recall, GB did best at lower folds, but RF became better at higher folds. To be specific, GB won accuracy for K = 5, and lost at K = 10 or 15. GB also won Recall for K = 5, 10, but not for K = 15. This indicates that RF overall works better at a higher amount of folds.

**Task 3 (40 points): Compare the effectiveness of the three models implemented above. Analyze
the results using the following:**

**A confusion matrix for one selected test fold**

The fold that I'm choosing for this is K = 5. I'll have to implement K-fold for CART, but that should pretty much be the same code as the other models.

K-fold for CART:

In [ ]:
CARTa = np.zeros(5)
CARTr = np.zeros(5)
CARTp = np.zeros(5)
for i, (train_index, test_index) in enumerate(skf5.split(X, y)):
  TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
  TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)

  Cartk = tfdf.keras.CartModel()
  Cartk.fit(TrainDatak)
  Cartk.compile(metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
  evaluation = Cartk.evaluate(TestDatak)
  CARTa[i] = evaluation[1]
  CARTr[i] = evaluation[2]
  CARTp[i] = evaluation[3]
CARTaccuracy = np.mean(CARTa)
CARTrecall = np.mean(CARTr)
CARTprecision= np.mean(CARTp)
print("accuracy:", CARTaccuracy)
print("recall:", CARTrecall)
print("precision", CARTprecision)

Use /tmp/tmp4vbvxgyq as temporary training directory
Reading training dataset...
Training dataset read in 0:00:02.563109. Found 491 examples.
Training model...
Model trained in 0:00:00.042274
Compiling model...


Model compiled.


1/1 [==============================] - 0s 483ms/step - loss: 0.0000e+00 - accuracy: 0.8537 - recall_5: 1.0000 - precision_5: 0.8252
Use /tmp/tmpxcp_njom as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.320764. Found 491 examples.
Training model...
Model trained in 0:00:00.049439
Compiling model...
Model compiled.
1/1 [==============================] - 0s 448ms/step - loss: 0.0000e+00 - accuracy: 0.7967 - recall_6: 0.9882 - precision_6: 0.7778
Use /tmp/tmp9n2nwgxk as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.269699. Found 491 examples.
Training model...
Model trained in 0:00:00.054048
Compiling model...
Model compiled.
1/1 [==============================] - 1s 594ms/step - loss: 0.0000e+00 - accuracy: 0.8130 - recall_7: 0.9762 - precision_7: 0.7961
Use /tmp/tmppjt_nlml as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.531646. Found 491 examples.
Training

The confusion matrix will be implemented on the 1st fold, since I can easily check for it using i % 5 = 0.

Up first is the CART model.

In [ ]:
for i, (train_index, test_index) in enumerate(skf5.split(X, y)):
  if i % 5 == 0:
    TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
    TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)
    Cartk = tfdf.keras.CartModel()
    Cartk.fit(TrainDatak)
    predictions = Cartk.predict(TestDatak)
    print(tf.math.confusion_matrix(y[test_index], tf.round(predictions)))



Use /tmp/tmpll2tc2hj as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.298955. Found 491 examples.
Training model...
Model trained in 0:00:00.043846
Compiling model...
Model compiled.
1/1 [==============================] - 0s 63ms/step
tf.Tensor(
[[20 18]
 [ 0 85]], shape=(2, 2), dtype=int32)


For the confusion matrix, the top left value is the number of true negatives, top right is false positives, bottom right is true positives and bottom left is false negatives. As we can see, the recall is perfect, but there's a slight inaccuracy with the precision.

Random forest:

In [ ]:
for i, (train_index, test_index) in enumerate(skf5.split(X, y)):
  if i % 5 == 0:
    TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
    TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)
    RandomForestk = tfdf.keras.RandomForestModel()
    RandomForestk.fit(TrainDatak)
    predictions = RandomForestk.predict(TestDatak)
    print(tf.math.confusion_matrix(y[test_index], tf.round(predictions)))



Use /tmp/tmpdkoe38kr as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.278001. Found 491 examples.
Training model...
Model trained in 0:00:00.489687
Compiling model...
Model compiled.
1/1 [==============================] - 0s 80ms/step
tf.Tensor(
[[21 17]
 [ 5 80]], shape=(2, 2), dtype=int32)


Here we see that an extra negative was guessed correctly, but at the cost of incorrectly labeling 5 positives. The recall and precision are both less than the CART model, I don't think the extra correct negative was worth it.

Finally, the Gradient Boost model:

In [ ]:
for i, (train_index, test_index) in enumerate(skf5.split(X, y)):
  if i % 5 == 0:
    TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
    TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)
    GradientBoostk = tfdf.keras.GradientBoostedTreesModel()
    GradientBoostk.fit(TrainDatak)
    predictions = GradientBoostk.predict(TestDatak)
    print(tf.math.confusion_matrix(y[test_index], tf.round(predictions)))



Use /tmp/tmpw74u7j9c as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.269926. Found 491 examples.
Training model...
Model trained in 0:00:00.330201
Compiling model...
Model compiled.
1/1 [==============================] - 0s 59ms/step
tf.Tensor(
[[21 17]
 [ 6 79]], shape=(2, 2), dtype=int32)


The negative values performed exactly the same as random forest, but another positive value was incorrectly identified. This was the worst performing of the algorithms in these circumstances.

**A statistical test (e.g., paired t-test) to determine if differences between models are
significant.**

I will be using a paired t-test. This will be done on the same fold as the confusion matrix. To do this, I will start by getting the predicted values for each model. For each pair, I will then take the differences between the values, and get the average. To get the standard error, I will take the standard deviation of the dataset and divide it by the square root of the amount of examples. Finally, I'll divide the average by the standard error to get a t value. Comparing this t value to the t table will determine how different these models are. For this table I'll use the p-value of 0.05. I used this website to calculate the table t value: https://datatab.net/tutorial/t-distribution

In [19]:
for i, (train_index, test_index) in enumerate(skf5.split(X, y)):
  if i % 5 == 0:
    TrainDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[train_index], label = label)
    TestDatak = tfdf.keras.pd_dataframe_to_tf_dataset(Database.loc[test_index], label = label)
    GradientBoostk = tfdf.keras.GradientBoostedTreesModel()
    GradientBoostk.fit(TrainDatak)
    predictionsg = GradientBoostk.predict(TestDatak)
    Cartk = tfdf.keras.CartModel()
    Cartk.fit(TrainDatak)
    predictionsc = Cartk.predict(TestDatak)
    RandomForestk = tfdf.keras.RandomForestModel()
    RandomForestk.fit(TrainDatak)
    predictionsr = RandomForestk.predict(TestDatak)

    GCdata = tf.round(predictionsg) - tf.round(predictionsc)
    GCavg = np.mean(GCdata)
    GCstd = np.std(GCdata)
    GCsterr = GCstd / np.sqrt(len(GCdata))
    print("Gradient Boost vs CART:")
    GCt = GCavg / GCsterr
    print(GCt)

    CRdata = tf.round(predictionsc) - tf.round(predictionsr)
    CRavg = np.mean(CRdata)
    CRstd = np.std(CRdata)
    CRsterr = CRstd / np.sqrt(len(CRdata))
    print("CART vs Random Forest:")
    CRt = CRavg / CRsterr
    print(CRt)

    RGdata = tf.round(predictionsr) - tf.round(predictionsg)
    RGavg = np.mean(RGdata)
    RGstd = np.std(RGdata)
    RGsterr = CRstd / np.sqrt(len(RGdata))
    print("Random Forest vs Gradient Boost:")
    RGt = RGavg / RGsterr
    print(RGt)

    print("table value: 2.228")


Use /tmp/tmpf3kd6bzr as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.264534. Found 491 examples.
Training model...
Model trained in 0:00:00.309613
Compiling model...
Model compiled.
1/1 [==============================] - 0s 70ms/step
Use /tmp/tmp9q076p9h as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.248326. Found 491 examples.
Training model...
Model trained in 0:00:00.042235
Compiling model...
Model compiled.
1/1 [==============================] - 0s 59ms/step
Use /tmp/tmp3ci6p1bd as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.340303. Found 491 examples.
Training model...
Model trained in 0:00:00.540723
Compiling model...
Model compiled.
1/1 [==============================] - 0s 79ms/step
Gradient Boost vs CART:
-2.7244109165741417
CART vs Random Forest:
2.5115120318503403
Random Forest vs Gradient Boost:
0.41858533864172337
table value: 2.228


The absolute value of the model differences is higher than the table value in the cases of Gradient Boost vs CART and CART vs Random Forest, so CART is significantly different from the other models. However, the t value is quite low for Random Forest vs Gradient Boost, so those models aren't significantly different.

**A discussion on the trade-off between bias and variance for each model.**

Overall the models were all similarly high on recall, and as such performed well with low bias. The worst performing was Random Forest at 5 folds, but that was still 93%, really good. This is also not the best for variance, as a high recall tends to correspond to a high variance. On the other hand, precision was around 80% for all the models, so that is a bit better for variance, and also better for bias. There's also the accuracy at a similar value and giving similar effects as precision. If I were to focus on one of the two to improve, it would be the variance, probably by adding more data points (if I had a way to collect them). It is likely another model type would be better for variance though, like a linear model.